In [1]:
import pandas as pd
import numpy as np
import requests

from collections import Counter

In [2]:
df_train = pd.read_csv("reviews_train.csv").drop("Unnamed: 0", axis=1)
df_test = pd.read_csv("reviews_test.csv").drop("Unnamed: 0", axis=1)

# Creating Feature Set #1

In [3]:
# Stop words
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

In [4]:
# Positive semantic words
url = "https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/positive-words.txt"
r = requests.get(url)
s=';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n; \n; Opinion Lexicon: Positive\n;\n; This file contains a list of POSITIVE opinion words (or sentiment words).\n;\n; This file and the papers can all be downloaded from \n;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html\n;\n; If you use this list, please cite one of the following two papers:\n;\n;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." \n;       Proceedings of the ACM SIGKDD International Conference on Knowledge \n;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, \n;       Washington, USA, \n;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing \n;       and Comparing Opinions on the Web." Proceedings of the 14th \n;       International World Wide Web conference (WWW-2005), May 10-14, \n;       2005, Chiba, Japan.\n;\n; Notes: \n;    1. The appearance of an opinion word in a sentence does not necessarily  \n;       mean that the sentence expresses a positive or negative opinion. \n;       See the paper below:\n;\n;       Bing Liu. "Sentiment Analysis and Subjectivity." An chapter in \n;          Handbook of Natural Language Processing, Second Edition, \n;          (editors: N. Indurkhya and F. J. Damerau), 2010.\n;\n;    2. You will notice many misspelled words in the list. They are not \n;       mistakes. They are included as these misspelled words appear \n;       frequently in social media content. \n;\n;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;'
pos_words = r.text[len(s)+2:]
pos_words = pos_words.split("\n")

In [5]:
# Negative semantic words
url = "https://raw.githubusercontent.com/jeffreybreen/twitter-sentiment-analysis-tutorial-201107/master/data/opinion-lexicon-English/negative-words.txt"
r = requests.get(url)
s=';;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n; \n; Opinion Lexicon: Negative\n;\n; This file contains a list of NEGATIVE opinion words (or sentiment words).\n;\n; This file and the papers can all be downloaded from \n;    http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html\n;\n; If you use this list, please cite one of the following two papers:\n;\n;   Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." \n;       Proceedings of the ACM SIGKDD International Conference on Knowledge \n;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, \n;       Washington, USA, \n;   Bing Liu, Minqing Hu and Junsheng Cheng. "Opinion Observer: Analyzing \n;       and Comparing Opinions on the Web." Proceedings of the 14th \n;       International World Wide Web conference (WWW-2005), May 10-14, \n;       2005, Chiba, Japan.\n;\n; Notes: \n;    1. The appearance of an opinion word in a sentence does not necessarily  \n;       mean that the sentence expresses a positive or negative opinion. \n;       See the paper below:\n;\n;       Bing Liu. "Sentiment Analysis and Subjectivity." An chapter in \n;          Handbook of Natural Language Processing, Second Edition, \n;          (editors: N. Indurkhya and F. J. Damerau), 2010.\n;\n;    2. You will notice many misspelled words in the list. They are not \n;       mistakes. They are included as these misspelled words appear \n;       frequently in social media content. \n;\n;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n\n'
neg_words = r.text[len(s):]
neg_words = neg_words.split("\n")

## Engineering the raw data

In [6]:
# Vectorizing reviews in training set
words_train = (
    df_train.Review.
    str.lower().
    str.replace("[^\w\s]","").
    str.split()
)
bag_of_words_train = words_train.apply(Counter)

In [7]:
# Vectorizing reviews in test set
words_test = (
    df_test.Review.
    str.lower().
    str.replace("[^\w\s]","").
    str.split()
)
bag_of_words_test = words_test.apply(Counter)

In [8]:
# Removing stop words in train
reviews_train = []
for r in words_train:
    good = []
    for w in r:
        if w not in en_stops:
            good.append(w)
    reviews_train.append(good)
reviews_train = pd.Series(reviews_train)

In [9]:
# Removing stop words in test
reviews_test = []
for r in words_test:
    good = []
    for w in r:
        if w not in en_stops:
            good.append(w)
    reviews_test.append(good)
reviews_test = pd.Series(reviews_test)

In [10]:
# Counting positive and negative words in training set
pos_set = set(pos_words)
neg_set = set(neg_words)
posc_train = []
negc_train = []
for r in reviews_train:
    count_pos = len(pos_set.intersection(set(r)))
    count_neg = len(neg_set.intersection(set(r)))
    posc_train.append(count_pos)
    negc_train.append(count_neg)
    

In [11]:
# Counting positive and negative words in test set
pos_set = set(pos_words)
neg_set = set(neg_words)
posc_test = []
negc_test = []
for r in reviews_test:
    count_pos = len(pos_set.intersection(set(r)))
    count_neg = len(neg_set.intersection(set(r)))
    posc_test.append(count_pos)
    negc_test.append(count_neg)

In [12]:
df_train = pd.DataFrame({"Positive_counts":posc_train, 
                         "Negative_counts":negc_train, 
                         "Label": df_train.Label})
df_train.head()

,Positive_counts,Negative_counts,Label
0,23,6,1
1,13,3,1
2,8,3,1
3,2,2,1
4,7,1,1


In [13]:
df_test = pd.DataFrame({"Positive_counts":posc_test, 
                        "Negative_counts":negc_test,
                        "Label": df_test.Label})
df_test.head()

,Positive_counts,Negative_counts,Label
0,13,6,1
1,2,0,1
2,7,8,1
3,6,3,1
4,6,2,1


## Saving Train and Test DataFrames

In [14]:
df_train.to_csv("train1.csv")
df_test.to_csv("test1.csv")